<a href="https://colab.research.google.com/github/neerajthandayan/Deep_Learning/blob/main/CNN_%2B_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network + GridSearch Cross-validation